In [44]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import the API key
from config import geoapify_key

In [45]:
honolulu_lat = 21.3099
honolulu_lon = -157.8581
limit = 100
radius = 8000
filters = f"circle:{honolulu_lon},{honolulu_lat},{radius}"
bias = f"proximity:{honolulu_lon},{honolulu_lat}"

params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "categories":"catering.restaurant",
    "apiKey":geoapify_key,
}

base_url = "https://api.geoapify.com/v2/places"

In [46]:
response = requests.get(base_url, params=params).json()
 
print(json.dumps(response, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -157.8599002,
                    21.30941810040861
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Catania Ristorante Italiano",
                "address_line2": "1111 Bishop Street, Honolulu, HI 96813, United States of America",
                "categories": [
                    "catering",
                    "catering.restaurant",
                    "catering.restaurant.italian"
                ],
                "catering": {
                    "cuisine": "italian"
                },
                "city": "Honolulu",
                "country": "United States",
                "country_code": "us",
                "county": "Honolulu County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database L

In [48]:
names =[]
types = []
addresses = []
distances = []
lats = []
longs = []

for f in response['features']:
    try:
        name = f["properties"]["name"]
        type = f['properties']['catering']['cuisine']
        address = f["properties"]["address_line2"]
        distance= f["properties"]["distance"]
        lat = f["properties"]["lat"]
        long = f["properties"]["lon"]
    except (KeyError, IndexError):
        type = "N/A"
        
    names.append(name)
    types.append(type)
    addresses.append(address)
    distances.append(distance)
    lats.append(lat)
    longs.append(long)

In [49]:
restaraunt_df = pd.DataFrame({'Name': names,
                                  'Type': types,
                                  'Address': addresses,
                                  'Distance(Meters)': distances,
                                  'Latitude':lats,
                                  'Longitude':longs})
restaraunt_df.head()

,Name,Type,Address,Distance(Meters),Latitude,Longitude
0,Catania Ristorante Italiano,italian,"1111 Bishop Street, Honolulu, HI 96813, United...",194,21.309418,-157.859900
1,Komala Curry House,indian,"700 Bishop Street, Honolulu, HI 96813, United ...",199,21.309369,-157.859937
2,Fort Street Cafe,vietnamese,"1152 Fort Street Mall, Honolulu, HI 96808, Uni...",236,21.310757,-157.860185
3,Red pepper,korean,"1150 Fort Street Mall, Honolulu, HI 96808, Uni...",241,21.310698,-157.860262
4,Sumo Sushi,sushi,"1300 Pali Highway, Honolulu, HI 96813, United ...",278,21.312394,-157.857804


In [50]:
restaraunt_df['Distance(Miles)'] = restaraunt_df['Distance(Meters)'] / 1609.34
restaraunt_df.head()

,Name,Type,Address,Distance(Meters),Latitude,Longitude,Distance(Miles)
0,Catania Ristorante Italiano,italian,"1111 Bishop Street, Honolulu, HI 96813, United...",194,21.309418,-157.859900,0.120546
1,Komala Curry House,indian,"700 Bishop Street, Honolulu, HI 96813, United ...",199,21.309369,-157.859937,0.123653
2,Fort Street Cafe,vietnamese,"1152 Fort Street Mall, Honolulu, HI 96808, Uni...",236,21.310757,-157.860185,0.146644
3,Red pepper,korean,"1150 Fort Street Mall, Honolulu, HI 96808, Uni...",241,21.310698,-157.860262,0.149751
4,Sumo Sushi,sushi,"1300 Pali Highway, Honolulu, HI 96813, United ...",278,21.312394,-157.857804,0.172742


In [51]:
restaraunt_df.to_json('restaurants.json', orient='records')